In [1]:
from rdkit import Chem                                                                                                                                                                   
from rdkit import RDLogger          
from rdkit.Chem import AllChem
from tqdm.auto import tqdm
       
import numpy as np                                                                                                                              
import os
from torch.utils.data import Dataset
import protein
import residue_constants
from datasets import LigProtDatabase

import torch
import torch.nn.functional as F



RDLogger.DisableLog('rdApp.*')                                                                                                                                                           


In [2]:

directory = '/Users/jihyeonje/Downloads/PDBBind_processed/'
ligpaths = []
protpaths = []

# iterate over files in
# that directory
for dir in os.listdir(directory):
    if dir !='.DS_Store':
        foldr = os.path.join(directory, dir)
    for i in os.listdir(foldr):
        if i.endswith('.sdf'):
            ligpaths.append(os.path.join(foldr, i))
        elif i.endswith('.pdb'):
            protpaths.append(os.path.join(foldr, i))

In [3]:
clean_ligpaths = []
clean_protpaths = []
for i in range(50):
    suppl = Chem.SDMolSupplier(ligpaths[i])
    if suppl is None: continue
    try:
        updated_mol= Chem.AddHs(suppl[0])
        AllChem.EmbedMolecule(updated_mol)
        clean_ligpaths.append(ligpaths[i])
        clean_protpaths.append(protpaths[i])
    except:
        print('fail')
        pass


fail
fail
fail
fail
fail
fail
fail


In [4]:
def make_train_valid_dfs(protpaths, ligpaths):
    max_id = len(protpaths)
    image_ids = np.arange(0, max_id)
    np.random.seed(42)
    valid_ids = np.random.choice(
        image_ids, size=int(0.2 * len(image_ids)), replace=False
    )
    train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    train_prot = list(map(protpaths.__getitem__, train_ids))
    train_ligs = list(map(ligpaths.__getitem__, train_ids))
    valid_prot = list(map(protpaths.__getitem__, valid_ids))
    valid_ligs = list(map(ligpaths.__getitem__, valid_ids))
    return train_prot, train_ligs, valid_prot, valid_ligs


In [5]:
train_prot, train_lig, valid_prot, valid_lig = make_train_valid_dfs(clean_protpaths, clean_ligpaths)

In [6]:
datas = LigProtDatabase(protpaths = train_prot, ligpaths = train_lig)
save_dir = f'/Users/jihyeonje/unidiffuser/test/feats/train'

In [9]:
for idx, data in tqdm(enumerate(datas)):
    if data is not None:
        prot, lig = data
        
        c_pad = torch.zeros(3, 1296 - prot.shape[1], 4)
        prot = torch.cat([prot, c_pad], dim=1)
    
        pad_lig = torch.zeros(100, lig.shape[1])
        pad_lig[:lig.shape[0], :] = lig
        prot = prot.detach().cpu().numpy()
        lig = pad_lig.detach().cpu().numpy()
        np.save(os.path.join(save_dir, f'{idx}.npy'), (prot,lig))



0it [00:00, ?it/s]

/Users/jihyeonje/unidiffuser/datasets.py:618: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ligand = torch.cat([torch.tensor(x), torch.tensor(one_hot)], dim=1)
/Users/jihyeonje/opt/anaconda3/envs/unidiffuser/lib/python3.9/site-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)
/Users/jihyeonje/unidiffuser/datasets.py:618: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ligand = torch.cat([torch.tensor(x), torch.tensor(o

In [8]:
datas = LigProtDatabase(protpaths = valid_prot, ligpaths = valid_lig)
save_dir = f'/Users/jihyeonje/unidiffuser/test/feats/test'

In [13]:
lig.shape

(100, 19)

In [18]:
import torch
import os
import numpy as np
import libs.autoencoder
import libs.clip
from datasets import MSCOCODatabase
import argparse
from tqdm import tqdm


def main():
    c = torch.zeros((100,19))
    save_dir = f'/Users/jihyeonje/unidiffuser/test/feats'
    c = c.detach().cpu().numpy()
    np.save(os.path.join(save_dir, f'empty_context.npy'), c)


if __name__ == '__main__':
    main()